## Week 6 Class activities
This notebook is a starting point for the exercises and activities that we'll do in class. We'll do an extension of the random forests classifier, looking at a continuous variable. Then, we'll do some cluster analysis.

Before you attempt any of these activities, make sure to watch the video lectures for this week.

### Classification: NYC evictions
We'll look at the factors that are associated with evictions in New York City. Perhaps a machine learning model can identify the types of places that are vulnerable to eviction, and target renter assistance programs more effectively?

#### Loading in the data

Let's start by loading in the [eviction dataset](https://data.cityofnewyork.us/City-Government/Evictions/6z8x-wfk4) via Socrata.

<div class="alert alert-block alert-info">

<strong>Exercise:</strong> Import the data from Socrata via the API into a pandas DataFrame.
</div>

*Hints*:
- Look back at Week 1 if you need a refresher on using Socrata
- There are about 70,000 rows in the dataset. So remember to add `?$limit=100000` to the end of the URL that you pass to `requests.get()`. Otherwise, you'll just get the first 1,000 rows. (The limit can be anything comfortably above 70000.)

In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd

# your code here

url = 'https://data.cityofnewyork.us/resource/6z8x-wfk4.json?$limit=100000'
r = requests.get(url)
df = pd.DataFrame(json.loads(r.content))
len(df)

<div class="alert alert-block alert-info">

<strong>Exercise:</strong> Convert your dataframe to a GeoDataFrame, using the latitude and longitude columns.

In [ ]:
# your code here 
gdf = gpd.GeoDataFrame(df, 
            geometry = gpd.points_from_xy(
                df.longitude, df.latitude, crs='EPSG:4326'))
gdf.plot() # make sure it looks ok

Now let's import some census data. We could use `cenpy` or the Census Bureau API. But to keep things simple so that we can focus on the spatial joins and the machine learning, I downloaded the block group-level 2019 ACS data for New York from the [Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-data.html). To save space, I clipped it to the 5 NYC counties.

It's in your repository, and we can load it in as follows. If you aren't familiar with a GeoPackage (GPKG) format, think of it as a "new and improved shapefile." [Here's a good overview.](https://towardsdatascience.com/why-you-need-to-use-geopackage-files-instead-of-shapefile-or-geojson-7cb24fe56416)

In [ ]:
bgs = gpd.read_file('data/nyc_bgs.gpkg')
bgs.head()

Note that the variables aren't particularly carefully selected - I just threw in many of the demographic and housing variables. 

Nor are the variable names particularly informative, but the full names are in a file in the repository.

In [ ]:
# note it is tab-sepated, not comma separated
# so we use the sep='\t' argument

col_names = pd.read_csv('data/BG_METADATA_2019.txt', sep='\t', index_col='Short_Name')
col_names.head()

So you can see the definition of the column like this. (I don't recommend renaming the `bg` column names, because the full names are so long.)

In [ ]:
col_names.loc['B01001e1']

#### Spatial join
Now let's do the spatial join. Again, let's follow our three step process.

1. Use a spatial join to add the `GEOID` column to the evictions dataframe. *Hint:* Check your projections.
2. Group by `GEOID` to get a count of evictions per block group. If you have a `Series`, give it a name - maybe `n_evictions`
3. Join those counts back - a tabular join based on the index

<div class="alert alert-block alert-info">
    <strong>Exercise:</strong> Add a count of evictions per census block group to your <strong>bgs</strong> GeoDataFrame, using the 3-step process above.
</div>

In [ ]:
# your code here

# spatial join
# match the CRS
bgs.to_crs('EPSG:4326', inplace=True)

gdf2 = gpd.sjoin(gdf, bgs, predicate='intersects')

# count 
n_evictions = gdf2.groupby('GEOID').size()
n_evictions.name = 'n_evictions'

# join back
bgs = bgs.set_index('GEOID').join(n_evictions)
bgs.n_evictions.fillna(0, inplace=True)
bgs.head()

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Do a quick-and-dirty map of the number of evictions. This will help identify any data holes.
</div>

In [ ]:
bgs.plot('n_evictions')

#### Random forests regressor
Now we have our data set. Let's estimate a random forests model.

In contrast to the previous examples, we are trying to predict a continuous variable - the number of evictions. So our classifier isn't appropriate. 

However, there is a similar model: the [random forest regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor). It works almost identically to the classifier. The main difference from a user perspective is assessing model performance - a confusion matrix doesn't work here.

You'll need to follow the following steps:
- choose your x variables. (Your y variable will be `n_evictions`)
- Drop Null values if needed
- split your dataset into training and testing portions
- estimate (fit) the model

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Estimate a random forest regressor model to predict the number of evictions per census tract.</div>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# your code here
# I'm just going to throw in all of the columns
xvars = [col for col in bgs.columns if col not in ['geometry','n_evictions']]
yvar = 'n_evictions'

# create a dataframe with no NaNs
df_to_fit = bgs[xvars+[yvar]].dropna()

# train test split
X_train, X_test, y_train, y_test = train_test_split(
    df_to_fit[xvars], df_to_fit[yvar], test_size = 0.25, random_state = 1)

# initialize the random forest classifer object
rf = RandomForestRegressor(n_estimators = 50, random_state = 1)

# now fit the model
rf.fit(X_train, y_train)

# make predictions
y_pred = rf.predict(X_test)


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Assess the fit of your model.</div>

Remember, the confusion matrix and accuracy scores don't apply here. Some ideas for continuous variables are [here](https://stackoverflow.com/questions/50789508/random-forest-regression-how-do-i-analyse-its-performance-python-sklearn). You could also plot actual vs predicted values.

In [ ]:
# your code here
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

# I used mean absolute error
print(metrics.mean_absolute_error(y_test, y_pred))

# but the scatter plot is more helpful to me
fig, ax = plt.subplots()
sns.regplot(x=y_test, y=y_pred, ax=ax)
ax.set_xlabel('Actual evictions')
ax.set_ylabel('Predicted evictions')

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Which variables are most important in your predictions? Plot the forest importances.</div>

In [ ]:
# your code here
# code from https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

import numpy as np
import seaborn as sns


importances = rf.feature_importances_

# convert to a series, and give the index labels from our X_train dataframe
forest_importances = pd.Series(importances, index=X_train.columns)

# get the standard deviations to be able to plot the error bars
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

# sort the importances in descending order
forest_importances.sort_values(inplace=True, ascending=False)

# let's plot just the top 10

fig, ax = plt.subplots(figsize=(4,10))
sns.barplot(x=forest_importances.values[:10], y=forest_importances.index[:10], yerr=std[:10], ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")

# let's find out what those variable are
# it would be better to actually change the axis tick label, but this is quick and dirty

for col_name in forest_importances.index.values[:10]:
    print(col_names.loc[col_name].values[0])

## Clustering
We'll now look at some k-means cluster analysis. The question: are there particular patterns of cruising for parking? You can see [my version of the analysis here](https://findingspress.org/article/28061-the-shape-of-cruising), joint with Robert Hampshire and Rachel Weinberger.

The data file that replicate the analysis is in your data folder. There is one row for each cruising trip (derived from the final portion of a GPS trace, once a driver is assumed to start looking for parking.)

You can load the data as follows.

In [ ]:
import pandas as pd
cruisingDf = pd.read_csv('data/cruising_shapes.csv')

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Replicate the analysis in Millard-Ball, Hampshire and Weinberger (2021). First, add each cluster label to the dataframe.</div>

In the paper, we used 5 clusters and the following columns: `'matchdist', 'frc_repeat', 'n_crossings', 'convexhull_ratio', 'frc_right', 'frc_left', 'frc_uturn', 'frc_straight', 'n_turns'`

* `matchdist` is path length (technically, the map-matched distance)
* `frc_repeat` is the fraction of repeated blocks (a driver drives on them more than once while cruising for parking)
* `n_crossings` is the number of times that the driver crosses over their path
* `convexhull_ratio` is a measure of the compactness of the search area
* `frc_right`, `frc_left`, `frc_uturn` and `frc_straight` are the fraction of times that the driver turns right or left, makes a U-turn, or continues straight at an intersection
* `n_turns` is the number of turns in the cruising trace

You'll need to:
* standardize the variables
* drop Null values
* run the k-means algorithm
* add the cluster labels back to your dataframe

How many observations do you get in each cluster?

In [ ]:
from sklearn.cluster import KMeans
from sklearn import preprocessing

# your code here
cols = ['matchdist', 'frc_repeat', 'n_crossings', 'convexhull_ratio', 'frc_right', 'frc_left', 'frc_uturn', 'frc_straight', 'n_turns']
scaler = preprocessing.StandardScaler().fit(cruisingDf[cols])
df_scaled = pd.DataFrame(scaler.transform(cruisingDf[cols]), 
                         columns=cols, index=cruisingDf.index)

kmeans = KMeans(n_clusters=5, random_state=1).fit(df_scaled)

df_scaled['cluster_id'] = kmeans.labels_
df_scaled.groupby('cluster_id').size()

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Create a radar plot of your clusters.</div>

Here's the function to create the radar chart, which we used in the video lecture.

It takes two arguments: the `kmeans` object created by `KMeans`, and your standardized dataframe.

In [ ]:
# code from https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D


def radar_factory(num_vars, frame='circle'):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'
        # use 1 line segment to connect specified points
        RESOLUTION = 1

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

def radar_plot(kmeans, df_scaled):
    N  = kmeans.cluster_centers_.shape[1]  # number of columns / variables
    k = kmeans.n_clusters
    theta = radar_factory(N, frame='polygon')
    data = kmeans.cluster_centers_.T
    spoke_labels = [col for col in df_scaled.columns if col!='cluster_id']
    fig, ax = plt.subplots(figsize=(9, 9),
                                subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.25, hspace=0.20, top=0.85, bottom=0.05)

    ax.plot(theta, data) #, color=color)
    ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    labels = ['Cluster {}'.format(kk) for kk in range(k)]
    ax.legend(labels, loc=(0.9, .95),
                                labelspacing=0.1, fontsize='small')
    
# your code here
radar_plot(kmeans, df_scaled)

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Experiment with different values of k (number of clusters), and using different columns.</div>

In [ ]:
# your code here

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> The dataframe comes with x and y coordinates. Use them to identify any spatial clusters of cruising trips in San Francisco.</div>

In [ ]:
# your code here
cols = ['x','y']

# let's not standardize, because we are working with latitude and longitude
# so the variables are already on (roughly) the same scale
#scaler = preprocessing.StandardScaler().fit(cruisingDf[cols])
#df_scaled = pd.DataFrame(scaler.transform(cruisingDf[cols]), 
#                         columns=cols, index=cruisingDf.index)

# restrict to San Francisco (otherwise we'll end up with a cluster in Ann Arbor)
sfDf = cruisingDf[cruisingDf.sf==True][cols].dropna()

# you can rerun this several times (through the map below) with different numbers of k
kmeans = KMeans(n_clusters=20, random_state=1).fit(sfDf)

sfDf['cluster_id'] = kmeans.labels_

In [ ]:
# these are the long/lat of our cluster centers
kmeans.cluster_centers_

In [ ]:
# create a geoDataFrame with just the geometry
import geopandas as gpd
centers = gpd.GeoDataFrame(geometry=
            gpd.points_from_xy(kmeans.cluster_centers_[:,0], 
                               kmeans.cluster_centers_[:,1]),
                          crs ='EPSG:4326')

# we could have done this in two steps, which might be easier to read
centers = pd.DataFrame(kmeans.cluster_centers_, columns=['lon','lat'])
centers = gpd.GeoDataFrame(geometry=
            gpd.points_from_xy(centers['lon'], 
                               centers['lat']),
                          crs ='EPSG:4326')

In [ ]:
# check it looks ok
centers.head()

In [ ]:
# create a geodataframe of the cruising points (so we can plot them as well)
cruisingGdf = gpd.GeoDataFrame(sfDf, 
                geometry=gpd.points_from_xy(sfDf.x, sfDf.y,
                    crs='EPSG:4326'))

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
fig, ax= plt.subplots(figsize=(10,10))

# plot the cluster centers with a large marker
centers.plot(ax=ax, markersize=100)

# plot the individual points with a small marker, and a different color for each cluster_id
cruisingGdf.plot(ax=ax, markersize=0.1, column='cluster_id')
ctx.add_basemap(ax=ax, crs='EPSG:4326', alpha=0.2)

<div class="alert alert-block alert-info">
<h3>What you should have learned</h3>
<ul>
  <li>Get more practice with spatial joins and Socrata.</li>
  <li>Learn how to estimate a random forests model for continuous data.</li>
  <li>Get more practice with standardizing data.</li>
  <li>Learn how to estimate a k-means cluster analysis.</li>
</ul>
</div>